<a href="https://colab.research.google.com/github/EduardoComparato/YouTrue/blob/main/YouTrue.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%pip -q install google-genai



In [ ]:
# Instalação pacote do Youtube Transcript
%pip -q install youtube-transcript-api

In [ ]:
# Configura a API Key do Google Gemini

import os
from google.colab import userdata

os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

In [ ]:
# Configura o cliente da SDK do Gemini

from google import genai

client = genai.Client()

MODEL_ID = "gemini-2.0-flash"

In [ ]:
# Pergunta ao Gemini uma informação mais recente que seu conhecimento

from IPython.display import HTML, Markdown

In [ ]:
# Instalar Framework de agentes do Google ################################################
!pip install -q google-adk

In [ ]:
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search
from google.genai import types  # Para criar conteúdos (Content e Part)
from datetime import date
import textwrap # Para formatar melhor a saída de texto
from IPython.display import display, Markdown # Para exibir texto formatado no Colab
import requests # Para fazer requisições HTTP
import warnings

warnings.filterwarnings("ignore")

In [ ]:
import uuid
from typing import Optional

# Função auxiliar que envia uma mensagem para um agente via Runner e retorna a resposta final
def call_agent(agent: Agent, message_text: str, user_id: Optional[str] = None, session_id: Optional[str] = None) -> str:
    # Cria um serviço de sessão em memória
    session_service = InMemorySessionService()

    # Gera user_id e session_id se não forem fornecidos
    user_id_to_use = user_id if user_id else str(uuid.uuid4())
    session_id_to_use = session_id if session_id else str(uuid.uuid4())
    # Cria uma nova sessão (você pode personalizar os IDs conforme necessário)
    session = session_service.create_session(app_name=agent.name, user_id=user_id_to_use, session_id=session_id_to_use)
    # Cria um Runner para o agente
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
    # Cria o conteúdo da mensagem de entrada
    content = types.Content(role="user", parts=[types.Part(text=message_text)])

    final_response = ""
    # Itera assincronamente pelos eventos retornados durante a execução do agente
    for event in runner.run(user_id=user_id_to_use, session_id=session_id_to_use, new_message=content):
        if event.is_final_response():
          if event.content is not None:
            for part in event.content.parts:
              if part.text is not None:
               final_response += part.text
               final_response += "\n"
    return final_response

In [ ]:
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [ ]:
##########################################
# --- Transcritor de Vídeos --- #
##########################################
def API_transcritor(ID_digitado):
    try:
        from youtube_transcript_api import YouTubeTranscriptApi, NoTranscriptFound, TranscriptsDisabled

        ytt_api = YouTubeTranscriptApi()
        transcricao = ytt_api.fetch(ID_digitado, languages=['pt-BR', 'en-US', 'pt', 'en','es'])

        return transcricao
    except NoTranscriptFound:
        print(f"No transcript found for video ID: {ID_digitado} in the specified languages or any other language.")
        return None
    except TranscriptsDisabled:
        print(f"Transcripts are disabled for video ID: {ID_digitado}.")
        return None
    except Exception as e:
        print(f"An error occurred while fetching the transcript for video ID {ID_digitado}: {e}")
        return None
    except TranscriptsDisabled:
        print(f"Transcripts are disabled for video ID: {ID_digitado}.")
        return None
    except Exception as e:
        print(f"An error occurred while fetching the transcript for video ID {ID_digitado}: {e}")
        return None

In [ ]:
################################################
# --- Agente 1: Informações do vídeo --- #
################################################
from googleapiclient.discovery import build

YOUTUBE_API_KEY = userdata.get("YOUTUBE_API_KEY")
youtube = build('youtube', 'v3', developerKey=YOUTUBE_API_KEY)

def agente_informativo(ID_digitado):
    try:
        request = youtube.videos().list(
            part="snippet,contentDetails",
            id=ID_digitado
        )
        response = request.execute()

        if response and 'items' in response and response['items']:
            video_info = response['items'][0]
            title = video_info['snippet']['title']
            duration = video_info['contentDetails']['duration']
            thumbnail = video_info['snippet']['thumbnails']['high']['url']

            return {
                "title": title,
                "duration": duration,
                "thumbnail": thumbnail
            }
        else:
            return "Vídeo não encontrado ou informações indisponíveis."
    except Exception as e:
        return f"Ocorreu um erro ao buscar as informações do vídeo: {e}"

In [ ]:
################################################
# --- Agente 2: Revisor de texto --- #
################################################
def agente_revisor(transcricao):
    revisor = Agent(
        name="agente_revisor",
        model="gemini-2.0-flash",
        instruction="""
        Você é um revisor de texto. Quero que pegue o texto mais recente fornecido pelo transcritor e retire elementos de minutagem e tópico,
        somente deixando o texto, separando em parágrafos ou não somente conforme for necessário. Caso o texto esteja em outra língua,
        sem ser o português. Traduza para o português.
        """,
        description="Agente que revisa textos",
    )

    entrada_do_agente_revisor = f"Texto a ser revisado: {transcricao}"
    # Executa o agente
    texto_revisado = call_agent(revisor, entrada_do_agente_revisor)
    return texto_revisado

In [ ]:
######################################
# --- Agente 3: Buscador e Primeiro Verificador --- #
######################################
def agente_buscador(texto_revisado):
    buscador = Agent(
        name="agente_buscador",
        model="gemini-2.0-flash",
        instruction="""
            Você é um buscador de fontes informacionais. A partir do texto fornecido pelo revisor,
            detecte o tema e pontos abordados e procure sobre em, NO MÁXIMO, 20 fontes confiáveis e
            de relevância no tema, onde as informações sejam por texto
            (evitando artigos de opinião e fontes muito imparciais).
            Enfatize, nessas fontes, as partes onde ocorram incongruências entre informações do texto e das fontes.
            Ao final, forneça os links das fontes pesquisadas.
            """,
        description="Agente que busca fontes relacionadas e faz uma primeira comparação",
        tools=[google_search]
    )
    entrada_do_agente_buscador = f"Texto revisado: {texto_revisado}"
    # Executa o agente
    busca_comparada = call_agent(buscador, entrada_do_agente_buscador)
    return busca_comparada

In [ ]:
##########################################
# --- Agente 4: Aponta-erros --- #
##########################################
def agente_erros(texto_revisado, rascunho_gerado):
    erros = Agent(
        name="agente_erros",
        model="gemini-2.0-flash",
        instruction="""
            Você aponta os erros de uma fonte informacional, tendo como base um texto e uma análise subsequente
            constando links de fontes confiáveis. Quero que vasculhe essas fontes, para confirmar se não há informações erradas no texto original.


            ###


            Com a análise de erros pronta, indique os trechos que estão errados, o porquê de estarem errados, e uma correção para tais trechos.
            Para cada correção, aponte a fonte que está sendo utilizada. Ao fim, indique a porcentagem do quanto o conteúdo do texto original estava
            comprometido por erros de informação.


            ###


            Ao fim, indique a confiabilidade do texto original a partir de 5 níveis, indicados por emojis,
            levando em consideração a porcentagem estabelecida anteriormente.

            Grau de confiabilidade:

            Confiável (0% a 7% de erros) -> 🟢

            Bom (8% a 20%) -> 🔵

            Mediano (20% a 45%) -> 🟡

            Ruim (46% a 60%) -> 🟠

            Grave (61% a 100%) -> 🔴

            ###

            Caso o texto final esteja em inglês, passe para o português.
            """,
        description="Agente que aponta erros, correções e grau de confiabilidade.",
        tools=[google_search]
    )
    entrada_do_agente_erros = f"Texto original: {texto_revisado}, Buscador: {busca_comparada}"
    # Executa o agente
    texto_corrigido = call_agent(erros, entrada_do_agente_erros)
    return texto_corrigido

In [ ]:
print("🚀 Iniciando o Sistema de Verificação de Informações para YouTube e textos transcritos com 4 Agentes 🚀")

# --- Função que detecta se o conteúdo é um link ou um texto ---
def obter_tipo_e_conteudo():
    """
    Oferece ao usuário a opção de inserir um link ou texto completo
    através de uma lista selecionável.

    Retorna:
        tuple: Uma tupla contendo o tipo de entrada ('link' ou 'texto')
               e o conteúdo inserido pelo usuário.
    """
    opcoes = ["ID", "Texto Completo"]
    print("Tipos de análises disponíveis: ")
    for i, opcao in enumerate(opcoes):
        print(f"{i + 1}. {opcao}")

    while True:
        try:
            escolha = int(input("Digite o número da sua escolha: "))
            if 1 <= escolha <= len(opcoes):
                tipo = "ID" if escolha == 1 else "texto"
                if tipo == "ID":
                  conteudo = input(f"🕵️ Insira o {opcoes[escolha - 1]} que deseja analisar: ")
                else:
                  conteudo = input(f"🕵️ Insira o {opcoes[escolha - 1].lower()} que deseja analisar: ")
                return tipo, conteudo
            else:
                print("Opção inválida. Por favor, digite um número da lista.")
        except ValueError:
            print("Entrada inválida. Por favor, digite um número.")

# Obtém o tipo de entrada e o conteúdo
tipo, conteudo = obter_tipo_e_conteudo()

# Agora você pode usar as variáveis 'tipo' e 'conteudo'
if tipo == "ID":
    ID_digitado = conteudo
    print(f"\nVocê inseriu um ID: {ID_digitado}")
    # Aqui você pode adicionar o código específico para lidar com links
elif tipo == "texto":
    texto_completo = conteudo
    print(f"\nVocê inseriu o seguinte texto:\n{texto_completo}")
    # Aqui você pode adicionar o código específico para lidar com o texto completo


####


# Inserir lógica do sistema de agentes ################################################
if tipo is None or conteudo is None:

    print("Infelizmente não podemos analisar o vazio!")

else:

    if tipo == "ID":

      print(f"Tudo bem! Vamos verificar as informações que constam em {conteudo}")

      print(f"As informações deste vídeo são: \n")

      print(agente_informativo(ID_digitado))

      print("--------------------------------------------------------------")

    else:

      print("Maravilha! Vamos então verificar o texto que você inseriu")

      print("--------------------------------------------------------------")


    if tipo == "ID":

      print("\n--- 📝 Resultado do Agente 1 (Transcritor) ---\n")

      transcricao = API_transcritor(ID_digitado)

      print("--------------------------------------------------------------")

    else:

      if transcricao is None:

          print("🚫 Não foi possível obter a transcrição. Encerrando o processo.")

      else:

        transcricao = texto_completo


    texto_revisado = agente_revisor(transcricao)

    print("\n--- 📝 Resultado do Agente 2 (Revisor) ---\n")

    display(to_markdown(texto_revisado))

    print("--------------------------------------------------------------")


    busca_comparada = agente_buscador(texto_revisado)

    print("\n--- 📝 Resultado do Agente 3 (Buscador) ---\n")

    display(to_markdown(busca_comparada))

    print("--------------------------------------------------------------")


    texto_corrigido = agente_erros(texto_revisado, busca_comparada)

    print("\n--- 📝 Resultado do Agente 4 (Erros) ---\n")

    display(to_markdown(texto_corrigido))

🚀 Iniciando o Sistema de Verificação de Informações para YouTube e textos transcritos com 4 Agentes 🚀
Tipos de análises disponíveis: 
1. ID
2. Texto Completo
Digite o número da sua escolha: 1
🕵️ Insira o ID que deseja analisar: Qd30j1C68OQ

Você inseriu um ID: Qd30j1C68OQ
Tudo bem! Vamos verificar as informações que constam em Qd30j1C68OQ
As informações deste vídeo são: 

{'title': 'Oscar Medina En la casa de mi padre', 'duration': 'PT3M26S', 'thumbnail': 'https://i.ytimg.com/vi/Qd30j1C68OQ/hqdefault.jpg'}
--------------------------------------------------------------

--- 📝 Resultado do Agente 1 (Transcritor) ---

--------------------------------------------------------------

--- 📝 Resultado do Agente 2 (Revisor) ---



> Existe um lugar, um lugar muito belo, de belas moradas onde Jesus Cristo me convida a morar. Ele disse: "Não temas, não vivas turbado, pois onde eu moro, ali tu comigo também vivarás."
> 
> Na casa de meu Pai há moradas para ti, na casa de meu Pai há moradas para ti, na casa de meu Pai há moradas para ti.
> 
> Minha paz vos deixo, esta paz de Deus e não como o mundo, o qual vai sem rumo, eu vos dou. E se eu me for preparar tudo, voltarei de novo para que comigo vós esteis.
> 
> Na casa de meu Pai há moradas para ti, na casa de meu Pai há moradas para ti, na casa de meu Pai há moradas para ti.
> 
> Na casa de meu Pai, moradas para ti.


--------------------------------------------------------------


KeyboardInterrupt: 